# Check for missing timesteps


In [1]:
import xarray as xr
import numpy as np 
import pandas as pd
import glob, os
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import multiprocessing

In [2]:
def round_down_to_nearest_5(year):
    return (year // 5) * 5

In [5]:
model="MPI-M.MPI-ESM1-2-LR"
# model="MIROC-ES2L"
# scen="ssp370"
# scen="ssp370_2049"
# scen="ssp245_2004"
# scen="ssp585_2004"
scen="ssp585_2049"

# year=2066
# year5=round_down_to_nearest_5(year)

# model="CanESM2"
# scen="rcp45"

# model="MIROC5"
# scen="historical"

# model="CCSM4"
# scen="rcp85"

# model="MRI-CGCM3"
# scen="rcp85"


In [7]:
# files  = sorted(glob.glob( f"/glade/derecho/scratch/bkruyt/CMIP6/WUS_icar_LivGrd3/{model}_{scen}/3hr/icar_3hr_livgrd_*"))
# files  = sorted(glob.glob( f"/glade/derecho/scratch/bkruyt/CMIP6/WUS_icar_LivGrd3/{model}_{scen}/3hr/icar_3hr_livgrd_*"))
# files  = sorted(glob.glob( f"/glade/campaign/ral/hap/bert/CMIP6/WUS_HUC12/bias_corrected_GMET_5yexl_detrendT_3/{model}_{scen}/bc_icar_*"))
files  = sorted(glob.glob( f"/glade/campaign/ral/hap/bert/CMIP6/WUS_icar_nocp_full/{model}_{scen}*/3hr/icar_3hr_*"))  

### CMIP5
# files  = sorted(glob.glob( f"/glade/campaign/ral/hap/bert/CMIP5/WUS_icar_livBC4/{model}_{scen}/3hr/icar_3hr_livgrid_*"))
# files  = sorted(glob.glob( f"/glade/campaign/ral/hap/bert/CMIP5/WUS_icar_nocp_full4/{model}_{scen}*/3hr/icar_3hr_*"))  

if len(files)==0 : print("no files found ")
else: print(f" {len(files)} files found")

### Loop through files and check the time diff 

In [9]:

for i in range(len(files)):
    f = files[i:i+2]  # open 2 files so we catch missing data at the end of a month
    # print(f)
    ds5 = xr.open_mfdataset( f )

    idx = np.where(ds5.time.diff(dim='time').values > ds5.time.diff(dim='time').min().values )
    # print(idx[0] )
    
    if len(idx[0])==1:
        print( #f.split('/')[-1], idx[0],
              model, scen, ds5.time.values[ idx[0][0] ] 
             )
    elif len(idx[0])>1:
        for l in range(len(idx[0])):
            print( 
                  model, scen, ds5.time.values[ idx[0][l] ] 
                 )
        

MPI-M.MPI-ESM1-2-LR ssp585_2049 2065-04-30T18:00:00.000000000
MPI-M.MPI-ESM1-2-LR ssp585_2049 2065-09-20T21:00:00.000000000
MPI-M.MPI-ESM1-2-LR ssp585_2049 2065-09-20T21:00:00.000000000


In [21]:
def check_mod_scen( model, scen, path):
    """Check for missing timesteps"""
    # files  = sorted(glob.glob( f"/glade/derecho/scratch/bkruyt/CMIP6/WUS_icar_LivGrd3/{model}_{scen}/3hr/icar_3hr_livgrd_*"))
    # files  = sorted(glob.glob( f"/glade/campaign/ral/hap/bert/CMIP6/WUS_HUC12/bias_corrected_GMET_5yexl_detrendT_3/{model}_{scen}/bc_icar_*"))
    if dt is not None:
        files  = sorted(glob.glob( f"{path}/{model}/{scen}/{dt}/icar_*"))
    else:
        files  = sorted(glob.glob( f"{path}/{model}/{scen}/icar_*"))

    for i in range(len(files)):
        f = files[i:i+2]  # open 2 files so we catch missing data at the end of a month
        
        ds5 = xr.open_mfdataset( f )
    
        idx = np.where(ds5.time.diff(dim='time').values > ds5.time.diff(dim='time').min().values )
        
        if len(idx[0])==1:
            print( #f.split('/')[-1], idx[0],
                  model, scen, ds5.time.values[ idx[0][0] ] 
                 )
        elif len(idx[0])>1:
            for l in len(idx[0]):
                print( 
                      model, scen, ds5.time.values[ idx[0][l] ] 
                     )
            

In [49]:
def check_Tmin_max( model, scen):

    # files  = sorted(glob.glob( f"/glade/derecho/scratch/bkruyt/CMIP6/WUS_icar_LivGrd3/{model}_{scen}/3hr/icar_3hr_livgrd_*"))
    # files  = sorted(glob.glob( f"/glade/campaign/ral/hap/bert/CMIP6/WUS_icar_nocp_full/{model}_{scen}/daily/icar_*"))
    files  = sorted(glob.glob( f"/glade/campaign/ral/hap/bert/CMIP6/WUS_HUC12/bias_corrected_GMET_5yexl_detrendT_3/{model}_{scen}/bc_icar_*"))

    for i in range(len(files)):
        f = files[i:i+2]  # open 2 files so we catch missing data at the end of a month
        
        ds5 = xr.open_mfdataset( f )
    
        # idx = np.where(ds5.time.diff(dim='time').values > ds5.time.diff(dim='time').min().values )

        cnt =  np.sum( ds5.Tmin.values > ds5.Tmax.values ) 
        print(model, scen, cnt)
        
        if cnt > 0: # if len(cnt[0]) > 0:
            idx = np.where( ds5.Tmin.values > ds5.Tmax.values )
            # print(f" Tmin> Tmax @ {ds5.time[idx[0][0] }")
            
            if len(idx[0])==1:
                print( #f.split('/')[-1], idx[0],
                      model, scen, ds5.time.values[ idx[0][0] ] 
                     )
            elif len(idx[0])>1:
                for l in len(idx[0]):
                    print( 
                          model, scen, ds5.time.values[ idx[0][l] ] 
                         )
            

### Serial

In [ ]:
models = ["CanESM5", "CMCC-CM2-SR5", "MIROC-ES2L", "MPI-M.MPI-ESM1-2-LR", "NorESM2-MM"] 
scenarios = ["ssp245_2004", "ssp245_2049", "ssp370_2004", "ssp370_2049", "ssp585_2004", "ssp585_2049"]

for model in models:
    for scen in scenarios:
        print(f"checking {model} {scen}:")
        check_mod_scen( model, scen )

### Parallel

In [19]:
models = ["CanESM5", "CMCC-CM2-SR5", "MIROC-ES2L", "MPI-M.MPI-ESM1-2-LR", "NorESM2-MM"] 
# scenarios = ["ssp245_2004", "ssp245_2049", "ssp370_2004", "ssp370_2049", "ssp585_2004", "ssp585_2049"]
scenarios = ["ssp245", "ssp370", "ssp585"]


with multiprocessing.Pool(processes=len(models)*len(scenarios)) as pool:
    results = pool.starmap(check_mod_scen, list(itertools.product(models, scenarios)))


In [ ]:
models = ["CanESM5", "CMCC-CM2-SR5", "MIROC-ES2L", "MPI-M.MPI-ESM1-2-LR", "NorESM2-MM"] 
# scenarios = ["ssp245_2004", "ssp245_2049", "ssp370_2004", "ssp370_2049", "ssp585_2004", "ssp585_2049"]
scenarios = ["ssp245", "ssp370", "ssp585"]


with multiprocessing.Pool(processes=len(models)*len(scenarios)) as pool:
    results = pool.starmap(check_Tmin_max, list(itertools.product(models, scenarios)))

In [50]:
check_Tmin_max( model, scen)

0
0


In [30]:
ds5 = xr.open_mfdataset( files[0] )


0

In [32]:
cnt = np.where( ds5.Tmin.values > ds5.Tmax.values ) 

In [35]:
len(cnt), len(cnt[0])

(2, 0)

In [37]:
cnt



(array([], dtype=int64), array([], dtype=int64))